In [ ]:
import pandas as pd

patient = pd.read_csv("data/eicu/patient.csv")
medication = pd.read_csv("data/eicu/medication.csv")
treatment = pd.read_csv("data/eicu/treatment.csv")
diagnosis = pd.read_csv("data/eicu/diagnosis.csv")

In [ ]:
eicu_time_148_281 = pd.read_csv("eicu_time_148_281.csv")
eicu_time_167_420 = pd.read_csv("eicu_time_167_420.csv")
eicu_time_199_458 = pd.read_csv("eicu_time_199_458.csv")
eicu_time_252_165 = pd.read_csv("eicu_time_252_165.csv")
eicu_time_449_283 = pd.read_csv("eicu_time_449_283.csv")

In [ ]:
eicu_dataset = pd.read_csv("eicu_dataset.csv")

In [ ]:
(eicu_time_199_458[(eicu_time_199_458["time_window"] == 0) & (eicu_time_199_458["ventilator"] == 1) & eicu_time_199_458[eicu_time_199_458['hospitalid'] == 199]]).shape[0] / eicu_time_199_458.shape[0]

In [ ]:
eicu_dataset.head()

In [ ]:
eicu_dataset[eicu_dataset["patientunitstayid"] == 534765.0]

In [ ]:
treatment.head()

In [ ]:
below_zero_count = (diagnosis['diagnosisoffset'] < 0).sum()
above_zero_count = (diagnosis['diagnosisoffset'] > 0).sum()

print(f"Number of rows with drugstartoffset below 0: {below_zero_count}")
print(f"Number of rows with drugstartoffset above 0: {above_zero_count}")

In [ ]:
medication[medication["patientunitstayid"] == 534765.0]

In [ ]:
treatment[treatment["patientunitstayid"] == 534765.0]

In [ ]:
diagnosis[(diagnosis["patientunitstayid"] == 534765.0) & (diagnosis['diagnosisstring']).str.contains("sepsis")]

In [ ]:
def calculate_unique_patients_counts(ventilator, bin_hour):
    ventilator_cp = ventilator.copy(deep=True)
    ventilator_cp['bin'] = (ventilator['treatmentoffset'] // (bin_hour * 60)).astype(int)
    
    # Count unique patients within each bin
    unique_patient_counts = ventilator_cp.drop_duplicates(subset=['patientunitstayid', 'bin']).groupby('bin').size()
    return unique_patient_counts

In [ ]:
ventilator = treatment[treatment['treatmentstring'].str.contains('ventilation')]
patient_df = patient[patient["hospitalid"] == 199]
hospital_patients = patient_df["patientunitstayid"].unique()
hospital_ventilator = ventilator[ventilator["patientunitstayid"].isin(hospital_patients)]
unique_patient_counts = calculate_unique_patients_counts(hospital_ventilator, 12)

In [ ]:
unique_patient_counts / len(patient_df)

In [ ]:
len(hospital_ventilator[(hospital_ventilator["treatmentoffset"] > 0) & (hospital_ventilator["treatmentoffset"] <= 720)]['patientunitstayid'].unique()) / len(patient_df)

In [ ]:
import pandas as pd
import fasttext
import numpy as np
from tqdm import tqdm
import re

unique_drugs = ['acetamin', 'biotene', 'compazine', 'ferrous', 'imdur', 'lidocaine', 'milk of magnesia', 'nystatin', 'prochlorperazine', 'tamsulosin',
                'advair diskus', 'bisacodyl', 'coreg', 'flagyl', 'influenza vac', 'lipitor', 'mineral', 'omeprazole', 'promethazine', 'thiamine',
                'albumin', 'bumetanide', 'cozaar', 'flomax', 'infuvite', 'lisinopril', 'mineral oil', 'ondansetron', 'propofol', 'ticagrelor',
                'albuterol', 'bumex', 'decadron', 'flumazenil', 'insulin', 'lispro', 'mono-sod', 'optiray', 'pulmicort respule', 'tiotropium',
                'allopurinol', 'buminate', 'definity', 'fluticasone-salmeterol', 'insulin detemir', 'loratadine', 'morphine', 'oxycodone', 'quetiapine', 'toradol',
                'alprazolam', 'calcium carbonate', 'deltasone', 'folic acid', 'iohexol', 'lorazepam', 'motrin', 'pantoprazole', 'refresh p.m. op oint', 'tramadol',
                'alteplase', 'calcium chloride', 'dexamethasone', 'furosemide', 'iopamidol', 'losartan', 'mupirocin', 'parenteral nutrition', 'reglan', 'trandate',
                'alum hydroxide', 'calcium gluconate', 'dexmedetomidine', 'gabapentin', 'ipratropium', 'maalox', 'nafcillin', 'percocet', 'restoril', 'transde rm-scop',
                'ambien', 'cardizem', 'dextrose', 'glargine', 'isosorbide', 'magnesium chloride', 'naloxone', 'phenergan', 'ringers solution', 'trazodone',
                'aminocaproic acid', 'carvedilol', 'diazepam', 'glucagen', 'kayciel', 'magnesium hydroxide', 'narcan', 'phenylephrine', 'rocuronium', 'ultram',
                'amiodarone', 'catapres', 'digoxin', 'glucagon', 'kayexalate', 'magnesium oxide', 'neostigmine', 'phytonadione', 'roxicodone', 'valium',
                'amlodipine', 'cefazolin', 'diltiazem', 'glucose', 'keppra', 'magnesium sulf', 'neostigmine methylsulfate', 'piperacillin', 'sennosides', 'vancomycin',
                'anticoagulant', 'cefepime', 'diphenhydramine', 'glycopyrrolate', 'ketorolac', 'magox', 'neurontin', 'plasmalyte', 'seroquel', 'vasopressin',
                'apresoline', 'ceftriaxone', 'diprivan', 'guaifenesin', 'klonopin', 'medrol', 'nexterone', 'plavix', 'sertraline', 'ventolin',
                'ascorbic acid', 'cephulac', 'docusate', 'haldol', 'labetalol', 'meperidine', 'nicardipine', 'pneumococcal', 'simethicone', 'vitamin',
                'aspart', 'cetirizine', 'dopamine', 'haloperidol', 'lactated ringer', 'meropenem', 'nicoderm', 'pnu-immune-23', 'simvastatin', 'warfarin',
                'aspirin', 'chlorhexidine', 'ecotrin', 'heparin', 'lactulose', 'merrem', 'nicotine', 'polyethylene glycol', 'sodium bicarbonate', 'xanax',
                'atenolol', 'ciprofloxacin', 'enoxaparin', 'humulin', 'lanoxin', 'metformin', 'nitro-bid', 'potassium chloride', 'sodium chloride', 'zestril',
                'atorvastatin', 'cisatracurium', 'ephedrine', 'hydralazine', 'lantus', 'methylprednisolone', 'nitroglycerin', 'potassium phosphate', 'sodium phosphate', 'zocor',
                'atropine', 'citalopram', 'epinephrine', 'hydrochlorothiazide', 'levaquin', 'metoclopramide', 'nitroprusside', 'pravastatin', 'polystyrene sulfonate', 'zolpidem',
                'atrovent', 'clindamycin', 'etomidate', 'hydrocodone', 'levemir', 'metoprolol', 'norco', 'precedex', 'spironolactone', 'zosyn',
                'azithromycin', 'clonazepam', 'famotidine', 'hydrocortisone', 'levetiracetam', 'metronidazole', 'norepinephrine', 'prednisone', 'sublimaze',
                'bacitracin', 'clonidine', 'fat emulsion', 'hydromorphone', 'levofloxacin', 'midazolam', 'normodyne', 'prilocaine', 'succinylcholine',
                'bayer chewable', 'clopidogrel', 'fentanyl', 'ibuprofen', 'levothyroxine', 'midodrine', 'norvasc', 'prinivil', 'tacrolimus']


def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def calculate_similarity(model, drug1, drug2):
    drug1_vector = model.get_word_vector(drug1)
    drug2_vector = model.get_word_vector(drug2)
    return cosine_similarity(drug1_vector, drug2_vector)


def get_age_group(age):

    if pd.isna(age):
        # 60 - 69 has the most patients
        return "60 - 69"

    if age == "> 89":
        return age

    age = int(age)

    if age < 30:
        return "< 30"
    elif age < 40:
        return "30 - 39"
    elif age < 50:
        return "40 - 49"
    elif age < 60:
        return "50 - 59"
    elif age < 70:
        return "60 - 69"
    elif age < 80:
        return "70 - 79"
    elif age < 90:
        return "80 - 89"
    else:
        return "> 89"


def get_bmi(weight_kg, height_cm):

    if pd.isna(weight_kg) or pd.isna(height_cm) or weight_kg == 0 or height_cm == 0:
        return "bmi_normal"

    height_m = height_cm / 100.0
    bmi = weight_kg / (height_m ** 2)

    if bmi < 18.5:
        return "bmi_underweight"
    elif bmi < 24.9:
        return "bmi_normal"
    elif bmi < 29.9:
        return "bmi_overweight"
    else:
        return "bmi_obesity"


def get_race(race):

    if pd.isna(race) or race == "Caucasian":
        return "race_caucasion"
    elif race == "African American":
        return "race_african"
    elif race == "Hispanic":
        return "race_hispanic"
    elif race == "Asian":
        return "race_asian"
    elif race == "Native American":
        return "race_native"
    else:
        return "race_caucasion"


def harmonize_drug(drug, unique_drugs, pretrained):

    if pd.isna(drug):
        return None

    drug = drug.lower()

    # Direct mapping: check whether drug names contain unique_drug
    for unique_drug in unique_drugs:
        if unique_drug in drug:
            return unique_drug

    # Check cosine similarity between word embeddings
    converted = re.sub(r'[^a-zA-Z\s]', '', drug)
    best_similarity = float('-inf')
    best_drug = None
    for drug_name in converted.split():
        for unique_drug in unique_drugs:
            similarity = calculate_similarity(
                pretrained, drug_name, unique_drug)
            if similarity > best_similarity:
                best_similarity = similarity
                best_drug = unique_drug

    return best_drug

import pandas as pd

patient = pd.read_csv("data/eicu/patient.csv")
medication_imputed = pd.read_csv("output/medication_imputed.csv")
diagnosis = pd.read_csv("data/eicu/diagnosis.csv")
treatment = pd.read_csv("data/eicu/treatment.csv")
patient_medication = patient.merge(medication_imputed, on='patientunitstayid', how='inner')
# patient_medication = patient_medication[patient_medication["hospitalid"].isin([167,420,199,458,252,165,148,281,449,283])]
patient_medication = patient_medication[patient_medication["hospitalid"].isin([199])]
ventilator_treatments = treatment[treatment['treatmentstring'].str.contains('ventilation', case=False, na=False)]
sepsis_diagnosis = diagnosis[diagnosis['diagnosisstring'].str.contains('sepsis', case=False, na=False)]

eicu_time = pd.DataFrame(columns=['patientunitstayid', 'hospitalid', 'time_window', 'ventilator', 'sepsis'] + unique_drugs + ['bmi_underweight', 'bmi_normal', 'bmi_overweight', 'bmi_obesity', 'race_african', 'race_hispanic', 'race_caucasion', 'race_asian', 'race_native', 'sex_is_male', 'sex_is_female', '< 30', '30 - 39', '40 - 49', '50 - 59', '60 - 69', '70 - 79', '80 - 89', '> 89'])

observation_time = 12 * 60
bin_time = 12 * 60
bin = observation_time // bin_time

# pretrained = fasttext.load_model("data/pretrained/BioWordVec_PubMed_MIMICIII_d200.bin")

# unique_drug_names = patient_medication['drugname'].unique()
# harmonized_drug_dict = {drug: harmonize_drug(drug, unique_drugs, pretrained) for drug in unique_drug_names}

patient = patient[patient["hospitalid"].isin([199])]
for index, row in tqdm(patient.iterrows(), total=patient.shape[0]):

    for cur_bin in range(bin):

        eicu_row = pd.DataFrame(0.0, index=[0], columns=eicu_time.columns)

        patientunitstayid = row['patientunitstayid']

        eicu_row['patientunitstayid'] = float(patientunitstayid)
        eicu_row['hospitalid'] = float(row['hospitalid'])
        eicu_row['time_window'] = cur_bin

        # BMI
        bmi = get_bmi(row['admissionweight'], row['admissionheight'])
        eicu_row[bmi] = 1.0

        # Race
        race = get_race(row['ethnicity'])
        eicu_row[race] = 1.0

        # Sex
        sex = "sex_is_female" if row['gender'] == 'Female' else "sex_is_male"
        eicu_row[sex] = 1.0

        # Age
        age_group = get_age_group(row['age'])
        eicu_row[age_group] = 1.0

        time_window_start = cur_bin * bin_time
        time_window_end = (cur_bin + 1) * bin_time

        # Label
        eicu_row["ventilator"] = float(len(ventilator_treatments[(ventilator_treatments["treatmentoffset"] >= time_window_start) & (ventilator_treatments["treatmentoffset"] < time_window_end) & (ventilator_treatments["patientunitstayid"] == patientunitstayid)]) > 0)

        eicu_row["sepsis"] = float(len(sepsis_diagnosis[(sepsis_diagnosis["diagnosisoffset"] >= time_window_start) & (sepsis_diagnosis["diagnosisoffset"] < time_window_end) & (sepsis_diagnosis["patientunitstayid"] == patientunitstayid)]) > 0)

        # Drug
        # time_window_drug = patient_medication[(patient_medication["patientunitstayid"] == patientunitstayid) & (patient_medication["drugstartoffset"] >= time_window_start) & (patient_medication["drugstartoffset"] < time_window_end)]

        # for drug in time_window_drug["drugname"]:
        #     
        #     converted_drug = harmonized_drug_dict.get(drug)
        #     if converted_drug is not None:
        #         eicu_row[converted_drug] = 1.0

        eicu_time = pd.concat([eicu_time, eicu_row], ignore_index=True)
# 
# print("Export time series eicu dataset...")
# eicu_time.to_csv('output/eicu_time_148_281.csv', index=False)
# eicu_time.head()


In [ ]:
ventilator_treatments[(ventilator_treatments["treatmentoffset"] > 0) & (ventilator_treatments["treatmentoffset"] <= 720)]['patientunitstayid'].unique()

In [ ]:
import pandas as pd
treatment = pd.read_csv("data/eicu/treatment.csv")
ventilator_treatments = treatment[treatment['treatmentstring'].str.contains('ventilation', case=False, na=False)]
len(ventilator_treatments[(ventilator_treatments["treatmentoffset"] >= 0) & (ventilator_treatments["treatmentoffset"] < 720) & (ventilator_treatments["patientunitstayid"] == 242040)])

In [ ]:
# treatment = pd.read_csv("data/eicu/treatment.csv")
# ventilator_treatments = treatment[treatment['treatmentstring'].str.contains('ventilation', case=False, na=False)]
# len(ventilator_treatments[(ventilator_treatments["treatmentoffset"] >= 0) & (ventilator_treatments["treatmentoffset"] < 720) & (ventilator_treatments["patientunitstayid"] == 242040)])

In [ ]:
diagnosis = pd.read_csv("data/eicu/diagnosis.csv")
sepsis_diagnosis = diagnosis[diagnosis["diagnosisstring"].str.contains("sepsis")]
len(sepsis_diagnosis[(sepsis_diagnosis["diagnosisoffset"] >= 0) & (sepsis_diagnosis["diagnosisoffset"] < 720) & (sepsis_diagnosis["patientunitstayid"] == 141227)]) > 0

In [ ]:
sepsis_diagnosis[(sepsis_diagnosis["diagnosisoffset"] > 0) & (sepsis_diagnosis["diagnosisoffset"] <= 720)]['patientunitstayid'].unique()

In [ ]:
# len(eicu_time[eicu_time["ventilator"] == 1.0]['patientunitstayid'].unique()) / len(eicu_time['patientunitstayid'].unique())

In [ ]:
# eicu_time.head()

In [ ]:
import pandas as pd

eicu_time_148_281 = pd.read_csv("eicu_time_148_281.csv")
eicu_time_167_420 = pd.read_csv("eicu_time_167_420.csv")
eicu_time_199_458 = pd.read_csv("eicu_time_199_458.csv")
eicu_time_252_165 = pd.read_csv("eicu_time_252_165.csv")
eicu_time_449_283 = pd.read_csv("eicu_time_449_283.csv")

In [ ]:
concatenated_df = pd.concat([eicu_time_148_281, eicu_time_167_420, eicu_time_199_458, eicu_time_252_165, eicu_time_449_283], ignore_index=True)
concatenated_df.to_csv("eicu_time_dataset.csv")

In [ ]:
# len(eicu_time_252_165[(eicu_time_252_165['time_window'] == 5) & (eicu_time_252_165['ventilator'] == 1) & (eicu_time_252_165['hospitalid'] == 252)]['patientunitstayid'].unique()) / len(eicu_time_252_165[ eicu_time_252_165['hospitalid'] == 252]['patientunitstayid'].unique())

In [ ]:
# [0.39038861 0.13022842 0.08899436 0.07208543 0.05487986 0.04983684]

In [ ]:
eicu_time = pd.read_csv("eicu_time_dataset.csv")

In [ ]:
filtered_df = eicu_time[eicu_time["patientunitstayid"] == 534765.0]

# Check which columns contain the value 1
columns_with_1 = filtered_df.apply(lambda x: x == 1).any()

# Print the columns that contain 1
print(columns_with_1[columns_with_1].index.tolist())

In [ ]:
eicu = pd.read_csv("eicu_dataset.csv")

In [ ]:
filtered_df = eicu[eicu["patientunitstayid"] == 534765.0]
# Check which columns contain the value 1
columns_with_1 = filtered_df.apply(lambda x: x == 1).any()

# Print the columns that contain 1
print(columns_with_1[columns_with_1].index.tolist())

In [ ]:
eicu.columns

In [ ]:
eicu_time.columns

In [ ]:
eicu_time['time_window']

In [ ]:
eicu_time

In [ ]:
eicu_time

In [ ]:
drug_columns = eicu_time.columns[6:-19]
sum_drugs_per_patient = eicu_time.groupby(['patientunitstayid', 'time_window'])[drug_columns].sum()
(sum_drugs_per_patient.sum(axis=1).groupby('patientunitstayid').mean()).mean()

# # Calculating the mean of these sums
# approx_drugs_per_patient = sum_drugs_per_patient.mean(axis=1)
# 
# # Print the result
# print(approx_drugs_per_patient)

In [ ]:
sum_drugs_per_patient = eicu_time.groupby('patientunitstayid').sum()
sum_drugs_per_time_window = df.groupby(['patientunitstayid', 'time_window'])[drug_columns].sum()

In [ ]:
sum_drugs_per_patient.iloc[:, 5:-19]

In [ ]:
eicu_time

In [ ]:
import pandas as pd

patient = pd.read_csv("data/eicu/patient.csv")
medication = pd.read_csv("data/eicu/medication.csv")

In [ ]:
patient.head()

In [ ]:
medication.head()

In [ ]:
medication_imputed.head()

In [ ]:
import pandas as pd
import fasttext
import numpy as np
import re

unique_drugs = ['acetamin', 'biotene', 'compazine', 'ferrous', 'imdur', 'lidocaine', 'milk of magnesia', 'nystatin', 'prochlorperazine', 'tamsulosin',
                'advair diskus', 'bisacodyl', 'coreg', 'flagyl', 'influenza vac', 'lipitor', 'mineral', 'omeprazole', 'promethazine', 'thiamine',
                'albumin', 'bumetanide', 'cozaar', 'flomax', 'infuvite', 'lisinopril', 'mineral oil', 'ondansetron', 'propofol', 'ticagrelor',
                'albuterol', 'bumex', 'decadron', 'flumazenil', 'insulin', 'lispro', 'mono-sod', 'optiray', 'pulmicort respule', 'tiotropium',
                'allopurinol', 'buminate', 'definity', 'fluticasone-salmeterol', 'insulin detemir', 'loratadine', 'morphine', 'oxycodone', 'quetiapine', 'toradol',
                'alprazolam', 'calcium carbonate', 'deltasone', 'folic acid', 'iohexol', 'lorazepam', 'motrin', 'pantoprazole', 'refresh p.m. op oint', 'tramadol',
                'alteplase', 'calcium chloride', 'dexamethasone', 'furosemide', 'iopamidol', 'losartan', 'mupirocin', 'parenteral nutrition', 'reglan', 'trandate',
                'alum hydroxide', 'calcium gluconate', 'dexmedetomidine', 'gabapentin', 'ipratropium', 'maalox', 'nafcillin', 'percocet', 'restoril', 'transde rm-scop',
                'ambien', 'cardizem', 'dextrose', 'glargine', 'isosorbide', 'magnesium chloride', 'naloxone', 'phenergan', 'ringers solution', 'trazodone',
                'aminocaproic acid', 'carvedilol', 'diazepam', 'glucagen', 'kayciel', 'magnesium hydroxide', 'narcan', 'phenylephrine', 'rocuronium', 'ultram',
                'amiodarone', 'catapres', 'digoxin', 'glucagon', 'kayexalate', 'magnesium oxide', 'neostigmine', 'phytonadione', 'roxicodone', 'valium',
                'amlodipine', 'cefazolin', 'diltiazem', 'glucose', 'keppra', 'magnesium sulf', 'neostigmine methylsulfate', 'piperacillin', 'sennosides', 'vancomycin',
                'anticoagulant', 'cefepime', 'diphenhydramine', 'glycopyrrolate', 'ketorolac', 'magox', 'neurontin', 'plasmalyte', 'seroquel', 'vasopressin',
                'apresoline', 'ceftriaxone', 'diprivan', 'guaifenesin', 'klonopin', 'medrol', 'nexterone', 'plavix', 'sertraline', 'ventolin',
                'ascorbic acid', 'cephulac', 'docusate', 'haldol', 'labetalol', 'meperidine', 'nicardipine', 'pneumococcal', 'simethicone', 'vitamin',
                'aspart', 'cetirizine', 'dopamine', 'haloperidol', 'lactated ringer', 'meropenem', 'nicoderm', 'pnu-immune-23', 'simvastatin', 'warfarin',
                'aspirin', 'chlorhexidine', 'ecotrin', 'heparin', 'lactulose', 'merrem', 'nicotine', 'polyethylene glycol', 'sodium bicarbonate', 'xanax',
                'atenolol', 'ciprofloxacin', 'enoxaparin', 'humulin', 'lanoxin', 'metformin', 'nitro-bid', 'potassium chloride', 'sodium chloride', 'zestril',
                'atorvastatin', 'cisatracurium', 'ephedrine', 'hydralazine', 'lantus', 'methylprednisolone', 'nitroglycerin', 'potassium phosphate', 'sodium phosphate', 'zocor',
                'atropine', 'citalopram', 'epinephrine', 'hydrochlorothiazide', 'levaquin', 'metoclopramide', 'nitroprusside', 'pravastatin', 'polystyrene sulfonate', 'zolpidem',
                'atrovent', 'clindamycin', 'etomidate', 'hydrocodone', 'levemir', 'metoprolol', 'norco', 'precedex', 'spironolactone', 'zosyn',
                'azithromycin', 'clonazepam', 'famotidine', 'hydrocortisone', 'levetiracetam', 'metronidazole', 'norepinephrine', 'prednisone', 'sublimaze',
                'bacitracin', 'clonidine', 'fat emulsion', 'hydromorphone', 'levofloxacin', 'midazolam', 'normodyne', 'prilocaine', 'succinylcholine',
                'bayer chewable', 'clopidogrel', 'fentanyl', 'ibuprofen', 'levothyroxine', 'midodrine', 'norvasc', 'prinivil', 'tacrolimus']


def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def calculate_similarity(model, drug1, drug2):
    drug1_vector = model.get_word_vector(drug1)
    drug2_vector = model.get_word_vector(drug2)
    return cosine_similarity(drug1_vector, drug2_vector)


def harmonize_drug(drug, unique_drugs, pretrained):

    if pd.isna(drug):
        return None

    drug = drug.lower()

    # Direct mapping: check whether drug names contain unique_drug
    for unique_drug in unique_drugs:
        if unique_drug in drug:
            return unique_drug

    # Check cosine similarity between word embeddings
    converted = re.sub(r'[^a-zA-Z\s]', '', drug)
    best_similarity = float('-inf')
    best_drug = None
    for drug_name in converted.split():
        for unique_drug in unique_drugs:
            similarity = calculate_similarity(
                pretrained, drug_name, unique_drug)
            if similarity > best_similarity:
                best_similarity = similarity
                best_drug = unique_drug

    return best_drug

# Selecting required columns
patient = pd.read_csv("data/eicu/patient.csv")
medication_imputed = pd.read_csv("output/medication_imputed.csv")

patient_medication = patient.merge(medication_imputed, on='patientunitstayid', how='inner')
patient_medication = patient_medication[patient_medication["hospitalid"].isin([199])]

selected_df = patient_medication[['patientunitstayid', 'hospitalid', 'drugstartoffset', 'drugname']]

# Applying the harmonize_drug function to drugname column
pretrained = fasttext.load_model("data/pretrained/BioWordVec_PubMed_MIMICIII_d200.bin")
selected_df['drugname'] = selected_df['drugname'].apply(lambda drug: harmonize_drug(drug, unique_drugs, pretrained))

# Sorting by patientunitstayid and then by drugstartoffset
sorted_df = selected_df.sort_values(by=['patientunitstayid', 'drugstartoffset'])
sorted_df.head()

In [ ]:
medication_imputed.shape

In [ ]:
import pandas as pd

eicu_time_comp565_148_281 = pd.read_csv("eicu_time_comp565_148_281.csv")
eicu_time_comp565_167_420 = pd.read_csv("eicu_time_comp565_167_420.csv")
eicu_time_comp565_199_458 = pd.read_csv("eicu_time_comp565_199_458.csv")
eicu_time_comp565_252_165 = pd.read_csv("eicu_time_comp565_252_165.csv")
eicu_time_comp565_449_283 = pd.read_csv("eicu_time_comp565_449_283.csv")

concatenated_df = pd.concat([eicu_time_comp565_148_281, eicu_time_comp565_167_420, eicu_time_comp565_199_458, eicu_time_comp565_252_165, eicu_time_comp565_449_283], ignore_index=True)
filtered_df = concatenated_df.dropna(subset=['drugname'])
filtered_df.to_csv("eicu_time_comp565.csv", index=False)

In [ ]:
pd.read_csv("eicu_time_comp565.csv")

In [ ]:
concatenated_df.dropna(subset=['drugname'])

In [ ]:
concatenated_df[concatenated_df['patientunitstayid'] == 534589].drugname.unique()

In [ ]:
concatenated_df.groupby('patientunitstayid')['drugname'].nunique()

# COMP 565

In [20]:
import pandas as pd

eicu_time_comp565 = pd.read_csv("eicu_time_comp565.csv")
eicu_time_comp565.head()

,patientunitstayid,hospitalid,drugstartoffset,drugname
0,534765,148,-3328,sodium chloride
1,534765,148,-3282,levofloxacin
2,534765,148,-3199,pantoprazole
3,534765,148,-3198,acetamin
4,534765,148,-3198,lorazepam


In [2]:
# eicu_time_comp565 = eicu_time_comp565.rename(columns={
#     'drugstartoffset': 'admissionoffset'
# })
# eicu_time_comp565.head()

,patientunitstayid,hospitalid,admissionoffset,drugname
0,534765,148,-3328,sodium chloride
1,534765,148,-3282,levofloxacin
2,534765,148,-3199,pantoprazole
3,534765,148,-3198,acetamin
4,534765,148,-3198,lorazepam


In [3]:
# hospitalid_map = eicu_time_comp565.set_index('patientunitstayid')['hospitalid'].to_dict()
# hospitalid_map

{534765: 148,
 534851: 148,
 535077: 148,
 535557: 148,
 535558: 148,
 535576: 148,
 535577: 148,
 535656: 148,
 535657: 148,
 535690: 148,
 535691: 148,
 535706: 148,
 535707: 148,
 535810: 148,
 535811: 148,
 536460: 148,
 536537: 148,
 536718: 148,
 536719: 148,
 537252: 148,
 537253: 148,
 537305: 148,
 537347: 148,
 537371: 148,
 537372: 148,
 537491: 148,
 537550: 148,
 538446: 148,
 538447: 148,
 538721: 148,
 538944: 148,
 538945: 148,
 538949: 148,
 538950: 148,
 539016: 148,
 539017: 148,
 539030: 148,
 539031: 148,
 539287: 148,
 539288: 148,
 539325: 148,
 539326: 148,
 539327: 148,
 539938: 148,
 539939: 148,
 539976: 148,
 540188: 148,
 540308: 148,
 540309: 148,
 540379: 148,
 540939: 148,
 541077: 148,
 541082: 148,
 541546: 148,
 541547: 148,
 541549: 148,
 541551: 148,
 541581: 148,
 541582: 148,
 542430: 148,
 542431: 148,
 542538: 148,
 542946: 148,
 542947: 148,
 542948: 148,
 543281: 148,
 543282: 148,
 543568: 148,
 543805: 148,
 543806: 148,
 544452: 148,
 54459

In [21]:
diagnosis = pd.read_csv("data/eicu/diagnosis.csv")
treatment = pd.read_csv("data/eicu/treatment.csv")
patient = pd.read_csv("data/eicu/patient.csv")
ventilator_treatments = treatment[treatment['treatmentstring'].str.contains('ventilation', case=False, na=False)]
ventilator_treatments = ventilator_treatments[ventilator_treatments["patientunitstayid"].isin(eicu_time_comp565["patientunitstayid"].unique())]
sepsis_diagnosis = diagnosis[diagnosis['diagnosisstring'].str.contains('sepsis', case=False, na=False)]
sepsis_diagnosis = sepsis_diagnosis[sepsis_diagnosis["patientunitstayid"].isin(eicu_time_comp565["patientunitstayid"].unique())]
mortality_patient = patient[patient["unitdischargestatus"] == 'Expired']
mortality_patient = mortality_patient[mortality_patient["patientunitstayid"].isin(eicu_time_comp565["patientunitstayid"].unique())]

In [22]:
sepsis_time = pd.DataFrame({
    'patientunitstayid': sepsis_diagnosis['patientunitstayid'],
    'sepsisdiagnosisoffset': sepsis_diagnosis['diagnosisoffset']
})
sepsis_time = sepsis_time.sort_values(by=['patientunitstayid', 'sepsisdiagnosisoffset'])
sepsis_time.to_csv("sepsis_comp565.csv", index=False)

In [23]:
sepsis_time.head()

,patientunitstayid,sepsisdiagnosisoffset
459454,534765,85
459456,534765,478
459498,534851,17
459499,534851,21
459712,535599,-13


In [24]:
ventilator_time = pd.DataFrame({
    'patientunitstayid': ventilator_treatments['patientunitstayid'],
    'ventilatortreatmentoffset': ventilator_treatments['treatmentoffset'],
})
ventilator_time = ventilator_time.sort_values(by=['patientunitstayid', 'ventilatortreatmentoffset'])
ventilator_time.to_csv("ventilator_comp565.csv", index=False)

In [25]:
ventilator_time.head()

,patientunitstayid,ventilatortreatmentoffset
932371,534621,121
932376,534765,85
932419,534930,588
932430,535142,531
932431,535142,1104


In [26]:
mortality_time = pd.DataFrame({
    'patientunitstayid': mortality_patient['patientunitstayid'],
    'deathoffset': mortality_patient['unitdischargeoffset'],
})
mortality_time = mortality_time.sort_values(by=['patientunitstayid', 'deathoffset'])
mortality_time.to_csv("mortality_comp565.csv", index=False)

In [27]:
mortality_time.head()

,patientunitstayid,deathoffset
29885,535599,67
30008,537671,14238
30105,538694,401
30108,538721,7250
30132,539163,18821


In [11]:
# eicu_time_comp565 = eicu_time_comp565.merge(sepsis_time, on=['patientunitstayid', 'admissionoffset'], how='outer')
# eicu_time_comp565 = eicu_time_comp565.merge(ventilator_time, on=['patientunitstayid', 'admissionoffset'], how='outer')
# eicu_time_comp565 = eicu_time_comp565.merge(mortality_time, on=['patientunitstayid', 'admissionoffset'], how='outer')

In [12]:
# eicu_time_comp565.head()

,patientunitstayid,hospitalid,admissionoffset,drugname,sepsis,ventilator,death
0,534765,148.0,-3328,sodium chloride,NaN,NaN,NaN
1,534765,148.0,-3282,levofloxacin,NaN,NaN,NaN
2,534765,148.0,-3199,pantoprazole,NaN,NaN,NaN
3,534765,148.0,-3198,acetamin,NaN,NaN,NaN
4,534765,148.0,-3198,lorazepam,NaN,NaN,NaN


In [13]:
# eicu_time_comp565['hospitalid'] = eicu_time_comp565['patientunitstayid'].map(hospitalid_map)
# eicu_time_comp565.head()

,patientunitstayid,hospitalid,admissionoffset,drugname,sepsis,ventilator,death
0,534765,148,-3328,sodium chloride,NaN,NaN,NaN
1,534765,148,-3282,levofloxacin,NaN,NaN,NaN
2,534765,148,-3199,pantoprazole,NaN,NaN,NaN
3,534765,148,-3198,acetamin,NaN,NaN,NaN
4,534765,148,-3198,lorazepam,NaN,NaN,NaN


In [14]:
# eicu_time_comp565 = eicu_time_comp565.sort_values(by=['hospitalid', 'patientunitstayid', 'admissionoffset'])
# eicu_time_comp565.head()

,patientunitstayid,hospitalid,admissionoffset,drugname,sepsis,ventilator,death
0,534765,148,-3328,sodium chloride,NaN,NaN,NaN
1,534765,148,-3282,levofloxacin,NaN,NaN,NaN
2,534765,148,-3199,pantoprazole,NaN,NaN,NaN
3,534765,148,-3198,acetamin,NaN,NaN,NaN
4,534765,148,-3198,lorazepam,NaN,NaN,NaN


In [15]:
# eicu_time_comp565['sepsis'].fillna(0, inplace=True)
# eicu_time_comp565['ventilator'].fillna(0, inplace=True)
# eicu_time_comp565['death'].fillna(0, inplace=True)

In [16]:
# eicu_time_comp565.head()

,patientunitstayid,hospitalid,admissionoffset,drugname,sepsis,ventilator,death
0,534765,148,-3328,sodium chloride,0.0,0.0,0.0
1,534765,148,-3282,levofloxacin,0.0,0.0,0.0
2,534765,148,-3199,pantoprazole,0.0,0.0,0.0
3,534765,148,-3198,acetamin,0.0,0.0,0.0
4,534765,148,-3198,lorazepam,0.0,0.0,0.0


In [ ]:
eicu_time_comp565.to_csv('output/eicu_time_comp565_2.csv', index=False)

In [ ]:
pd.read_csv("output/eicu_time_comp565_2.csv")